<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

# Point-Repeat Example using waveforms 

The scirpt demonstrates the usage of ptRepeat option to enable effective lower signal generation frequencies.

The scpi command: ':PTRepeat X1|X2|X4|X8' defines the factor of the generation slow down. This is done by repeating each data-point during the generation in accordance to the parameter set (Default value is X1). The PTRepeat function can work by using the slow segments which are defined from Segment # 250 onwards. Also, note that PTRepeat parameters can be set per-channel independtly only if each channel generates from separate segments.


### set up:

* Proteues device P94xx and connect channel 1 to the scope.
* This scripts works either the PXI or Lan interfaces.
* Connect any of P94xx channels to Scope input to verfigy test results. 

### Proteus version: 
Anabelle

In [25]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [26]:
import os
import gc
import sys
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst

### Before we start

This example is designed to work both when there is a LAN based communication-service (usually on benchtop/desktop)<br>
And when there's no communication-service, and the user's application has access to hardware-modules in the PXI chassis.

In the first case, when there's a LAN communication, this example uses the class `TEVisaInst` from `tevisainst`.<br>

In the second case, this example uses the classes `TEProteusAdmin` and `TEProteusInst` from `teproteus`.

In [27]:
# Connect to instrument

inst = None
admin = None

conn_msg_str = ''

def disconnect():
    global inst
    global admin
    if inst is not None:
        try:
            inst.close_instrument()            
        except:
            pass
        inst = None
    if admin is not None:
        try:
            admin.close_inst_admin()
        except:
            pass
        admin = None
    gc.collect()

def connect_to_pxi_slot(slot_id):
    global inst
    global admin
    try:
        disconnect()        
        admin = TEProteusAdmin()
        admin.open_inst_admin()        
        inst = admin.open_instrument(slot_id, reset_hot_flag=True)
    except:
        pass

def connect_to_lan_server(ip_address):
    global inst
    try:
        disconnect()
        inst = TEVisaInst(ip_address, port=5025)
    except:
        pass

def connect(ipaddr_or_slotid):
    try:
        disconnect()
        if isinstance(ipaddr_or_slotid, str) and '.' in ipaddr_or_slotid:
            connect_to_lan_server(ipaddr_or_slotid)
        else:
            connect_to_pxi_slot(ipaddr_or_slotid)
    except:
        pass

addr_textbox = widgets.Text(
    value='',
    placeholder='IP Address or Slot Id',
    description='Address:',
    disabled=False
)

output = widgets.Output()

def print_conn_message(msg_str):
    global output
    global conn_msg_str
    with output:
        if len(conn_msg_str) > 0:
            print('\r', end='')
            print(' ' * len(conn_msg_str), end='')
            print('\r', end='')
        conn_msg_str = str(msg_str)
        if len(conn_msg_str) > 0:
            print(conn_msg_str, end='')

def on_button_connect_clicked(b):
    global inst
    global addr_textbox
    print_conn_message('')
    disconnect()
    if addr_textbox is not None:
        conn_str = addr_textbox.value
        if conn_str:
            print_conn_message('Connecting to {0} ..'.format(conn_str))
            connect(conn_str)
            print_conn_message('')
    if inst is not None:
        idn_str = inst.send_scpi_query('*IDN?')
        if idn_str:
            print_conn_message('Connected to: ' + idn_str.strip())
    
def on_button_disconnect_clicked(b):
    print_conn_message('')
    disconnect()

btn_connect = widgets.Button(description="Connect")
btn_disconnect = widgets.Button(description="Disconnect")

btn_connect.on_click(on_button_connect_clicked)
btn_disconnect.on_click(on_button_disconnect_clicked)

display(addr_textbox, btn_connect, btn_disconnect, output)

Text(value='', description='Address:', placeholder='IP Address or Slot Id')

Button(description='Connect', style=ButtonStyle())

Button(description='Disconnect', style=ButtonStyle())

Output()

### Paranoia Level

When sending a SCPI command to instrument, it is highlly recommended to send it as query with `*OPC?`.<br>
For example, instead of sending bare SCPI command `:OUTPUT ON`, send a compound query `:OUTPUT ON; *OPC?`<br>
In that way the user makes sure the execution of the command has been completed before sending the next command.

The class `TEProteusInst` from `teproteus` and the class `TEVisaInst` from `tevisainst` both have a method<br>
called `send_scpi_cmd(scpi_str, paranoia_level=None)` that sends the given SCPI string `scpi_str` to the instrument.<br>

The optional `paranoia_level` argument can receive the values 0, 1 or 2 where
 - 0 means: send `scpi_str` as a bare SCPI command.
 - 1 means: append `*OPC?` to the given `scpi_str`, send it as query, and read the response (which is "1").
 - 2 means: append `:SYST:ERR?` to the given, send it as query and read the response (the last system error).

If the optional `paranoia_level` argument is not given then a default paranoia-level (that the user can setup) is used.

The initial value of the default paranoia-level is 1.<br>
High paranoia-level (2) is good for debugging, because the system-error is checked after each SCPI command.     

In [28]:
connect("192.168.0.117")

#connect("5")

if inst is not None:
        idn_str = inst.send_scpi_query('*IDN?')
        model = inst.send_scpi_query(":SYST:iNF:MODel?")
        if idn_str:
            prnmsg = 'Connected to: ' + idn_str.strip()
            print(prnmsg)
else:
    print("Not connected")

pid = os.getpid()
print('process id {0}'.format(pid))


# Set the default paranoia-level (0, 1 or 2)
inst.default_paranoia_level = 2 # good for debugging

# Clear error-list and reset the instrument
inst.send_scpi_cmd('*CLS; *RST')

Connected to: Tabor Electronics,P9082M,000000000000,1.235.0 --slot#: 5,
process id 28948


0

In [29]:
# Get the model name
model = inst.send_scpi_query(":SYST:iNF:MODel?")
print("Model: " + model)

# Infer the natural DAC waveform format
if 'P908' in model:
    dac_mode = 8
else:
    dac_mode = 16
    


print("DAC waveform format: {0} bits-per-point".format(dac_mode))

max_dac = 2 ** dac_mode - 1
half_dac = max_dac / 2.0
print('Max DAC wave-point level: {0}'.format(max_dac))
print('Half DAC wave-point level: {0}'.format(half_dac))

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))


Model: P9082M
DAC waveform format: 8 bits-per-point
Max DAC wave-point level: 255
Half DAC wave-point level: 127.5
Number of channels: 2
Max segment number: 65536
Available memory per DDR: 4,286,560,192 wave-bytes


## Set DAC frequency

In [30]:
FREQ = 1000
inst.send_scpi_cmd(":FREQ:RAST {0}MHz".format(FREQ))
resp = inst.send_scpi_query(":FREQ:RAST?")
freq = float(resp)
print ("DAC Generate Freq:{0}".format(freq))

DAC Generate Freq:1000000000.0


## Writing waveform data to segment

The command `:TRACE:DATA [<offset-in-bytes>] #<binary-header><binary-data>`  
writes the given block of binary data to the specified offset in the selected segment of the selected channel.

#### The header of the binary-data block
The SCPI protocol uses text-based messages, and the new-line character denotes end of a SCPI statement.<br>
In case of binary-data the sender must send a header that defines the size of the binary block before the data  
because the new-line character can be a part of the data.<br>

The header is composed of `'#'` character followed by a sequence of between 2 and 10 decimal digits, where the<br>
first digits denotes how many digits follows it (between 1 and 9) and the rest of the digits denote the size of the<br>
binary-block in bytes. For example, `#3256` denotes block of 256 bytes, and `#41024` denotes block of 1024 bytes.

### Simple example

Download waves to several segments and play different segment in each channel.  


In [31]:
# Build 12 waveforms

seglen = 81920 #2**28
num_cycles = (1, 2, 4, 8)  #(2^16,0)

waves = [ None for _ in range(1)]

for ii in range(1):
    ncycles = num_cycles[ii]
    cyclelen = seglen / ncycles
    
    x = np.linspace(
        start=0, stop=2 * np.pi * ncycles, num=seglen, endpoint=False)
    y = (np.sin(x) + 1.0) * half_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    if dac_mode == 16:
        waves[ii] = y.astype(np.uint16)
    else:
        waves[ii] = y.astype(np.uint8)
    

## Define and download segments

In [32]:
segList = []
# Download segments
for ii in range(num_channels):
    ichan = ii
    channb = ichan + 1
    segnum = ichan % 2 + 250
    segList.append(segnum)
    wav = waves[0] #ichan]
    print('Download wave to segment {0} of channel {1}'.format(segnum, channb))
    
    # Select channel
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd)
    
    # Define segment
    cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
    inst.send_scpi_cmd(cmd)
    
    # Select the segment
    cmd = ':TRAC:SEL {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)
        
    # Send the binary-data:
    inst.write_binary_data(':TRAC:DATA', wav)
        
    resp = inst.send_scpi_query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{0}" after writing binary values'.format(resp))

    # Play the specified segment at the selected channel:
    cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)

    # Turn on the output of the selected channel:
    inst.send_scpi_cmd(':OUTP ON')
    
    resp = inst.send_scpi_query(':SYST:ERR?')
    print(resp)

print()

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

del waves
del wav

Download wave to segment 250 of channel 1
0, no error
Download wave to segment 251 of channel 2
0, no error

0, no error


## Seting PTRepeat Parameters Per Channel


* Set the tested channel with the :
** 'ichan=1' assignment.
* Set PtRepeat paramter using the : 
** 'ptX=2' assignemt

Connect any of P94xx channels to Scope input to verfigy test results. 

In [33]:
ichan = 1
ptX=1

cmd = ':INST:CHAN {0}'.format(ichan)
inst.send_scpi_cmd(cmd)
inst.send_scpi_cmd(':PTRepeat X{0}'.format(ptX))

0

## Build task table

Build 2 task, and toggle between them for endless loop

Connect any of P94xx channels to Scope input to verfigy test results. 

In [34]:
tasklen = 2

for ii in range(num_channels):
    channb=ii+1
    #Select channel
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd )
    
    cmd = ':TASK:COMP:LENG {0}'.format(tasklen)
    inst.send_scpi_cmd(cmd )
    
    for jj in range(tasklen):
        curr_task = jj+1
        loop = jj+1
        segnb = jj+1
        cmd = ':TASK:COMP:SEL {0}'.format(curr_task)
        inst.send_scpi_cmd(cmd )
       
        cmd = ':TASK:COMP:TYPE SING'
        inst.send_scpi_cmd(cmd )
       
        cmd = ':TASK:COMP:LOOP {0}'.format(loop)
        inst.send_scpi_cmd(cmd )
       
        cmd = ':TASK:COMP:SEGM {0}'.format(segnb)
        inst.send_scpi_cmd(cmd )
        
        if curr_task==tasklen:
            cmd = ':TASK:COMP:NEXT1 {0}'.format(1)
            inst.send_scpi_cmd(cmd)
        else:
            cmd = ':TASK:COMP:NEXT1 {0}'.format(curr_task+1)
            inst.send_scpi_cmd(cmd )
    
    cmd = ':TASK:COMP:WRIT'
    inst.send_scpi_cmd(cmd )
    print('Downloading Task table of channel {0}'.format(channb))
    
    resp = inst.send_scpi_query(':SYST:ERR?')
    print(resp)

0, no error
0, no error


## Check frequency

In [35]:
cmd = 'FUNC:MODE TASK'
inst.send_scpi_cmd(cmd)

dac_mode = 16
#inst.send_scpi_cmd(':SOUR:FREQ 2.5e9')
resp = inst.send_scpi_query(':SOUR:FREQ?')
print("Frequency: {}".format(resp))
print(dac_mode)
    

Frequency: 1000000000.000
16


## Power on to all channels

In [36]:
# Change the segment that being played in channel 1,
# and play all segments one after the other with some time-delay

for ichan, iseg in enumerate(segList):    
     # Select channel
    print(f'ichan={ichan}, iseg={iseg}')
    cmd = ':INST:CHAN {0}'.format(ichan+1)
    inst.send_scpi_cmd(cmd)
    
    #cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(iseg)
    #inst.send_scpi_cmd(cmd)
    
    inst.send_scpi_cmd(':OUTP ON')
    time.sleep(0.2)   


ichan=0, iseg=250
ichan=1, iseg=251


In [13]:
# Disconnect
print_conn_message('')
disconnect()